In [1]:
conda install selenium

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [74]:
import numpy as numpy
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import datetime
import time

def lazyloading(driver):
    # Get the current number of rows
    current_rows_number = len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0'))
    total_number = int(driver.find_elements_by_css_selector('.querySummary__querySummary--39WP2')[0].text.split('Showing ')[1].split('Sparkling wines')[0])
    print("Total Number: ", total_number)
    while current_rows_number < total_number:
        # Scroll down to make new XHR (request more table rows)
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        try:
            driver.set_window_position(0,-1000)
            # Wait until number of rows increased
            time.sleep(10)
            wait(driver, 20).until(lambda drive: len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0')) > current_rows_number)
            # Update variable with current rows number
            current_rows_number = len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0'))
            print("Curr Row Number: ", current_rows_number)
        # If number of rows remains the same after 5 seconds passed, break the loop
        # as there no more rows to receive
        except TimeoutException:
            break

In [ ]:
from wine_setup import WineSetup

sparkling_wine_url = "https://www.vivino.com/explore?e=eJwNizsKgDAQBW_z6ohYvs4biJWIrDGGgFHJxt_tTTNTDBMTK8Sw0yDKy8YY2I99B1vQ4izVr7wlBZdlwzEzSQ6710lul8Q7HFycWjx5GFmXWYsvhdLKD5MkH4Q="
sparkling_wine = Wine(sparkling_wine_url)
sparkling_wine.setup_driver()
sparkling_wine.run_timestamp()
lazyloading(sparkling_wine.driver)
sparkling_wine.run_timestamp()

Current Timestamp:  05-Feb-2020 (19:31:37.142036)
Total Number:  1195
Curr Row Number:  50
Curr Row Number:  75
Curr Row Number:  100


In [70]:
sparkling_html = BeautifulSoup(sparkling_wine.driver.page_source, 'lxml')
sparkling_div = sparkling_html.find("div", {"class": "explorerPage__results--3wqLw"})
sparkling_rows = sparkling_html.find_all("div", {"class": "explorerCard__explorerCard--3Q7_0"})

In [71]:
import copy

# austria_wine_df = pd.DataFrame()
all_rows = []

# Let's store each row as a dictionary 
empty_row = {
    "title": None, "location": None, "country": None, "price": None, "type": None, "ratings": None, "num_ratings": None, "reviews": None, "url": None
}

for row in sparkling_rows:
    new_row = copy.copy(empty_row)
    # A list of all the entries in the row.
    new_row['title'] = row.find("span", {"class": "vintageTitle__wine--U7t9G"}).text
    location = row.find("div", {"class": "vintageLocation__vintageLocation--1DF0p"})
    new_row['location'] = location.findChildren()[-1].text
    country = row.find("i", {"class": "vintageLocation__countryFlag--1HbXr"})['title']
    new_row['country'] = country
    price_button = row.find("button", {"class": "addToCartButton__addToCartButton--qZv9F"})
    if price_button:
        new_row['price'] = (float(price_button.find("span").text.replace("$", "")))
    new_row['type'] = 'Sparkling'
    new_row['ratings'] = row.find("div", {"class": "vivinoRatingWide__averageValue--1zL_5"}).text
    new_row['num_ratings'] = int(row.find("div", {"class": "vivinoRatingWide__basedOn--s6y0t"}).text.split()[0])
    review_div = row.find("div", {"class": "review__note--2b2DB"})
    if review_div:
        new_row['reviews'] = review_div.text
    clean_div = row.find("div", {"class": "cleanWineCard__cleanWineCard--tzKxV cleanWineCard__row--CBPRR"})
    if clean_div:
        new_row['url'] = 'https://www.vivino.com' + clean_div.find("a")['href']
    all_rows.append(new_row)

In [67]:
all_rows

[{'title': 'Brut Champagne N.V.',
  'location': 'Champagne',
  'region': 'Champagne',
  'country': 'France',
  'price': 177.94,
  'type': 'Sparkling',
  'ratings': '4.6',
  'num_ratings': 87993,
  'reviews': None,
  'image_url': 'images.vivino.com/thumbs/s5aXYaQiTu-V_xEYI3KXRg_pb_x300.png'},
 {'title': 'Cristal Brut Champagne (Millésimé) N.V.',
  'location': 'Champagne',
  'region': 'Champagne',
  'country': 'France',
  'price': 247.19,
  'type': 'Sparkling',
  'ratings': '4.6',
  'num_ratings': 24412,
  'reviews': None,
  'image_url': 'images.vivino.com/thumbs/s5aXYaQiTu-V_xEYI3KXRg_pb_x300.png'},
 {'title': 'Grande Cuvée Brut Champagne N.V.',
  'location': 'Champagne',
  'region': 'Champagne',
  'country': 'France',
  'price': 177.97,
  'type': 'Sparkling',
  'ratings': '4.6',
  'num_ratings': 14411,
  'reviews': None,
  'image_url': 'images.vivino.com/thumbs/s5aXYaQiTu-V_xEYI3KXRg_pb_x300.png'},
 {'title': 'Brut Champagne 2008',
  'location': 'Champagne',
  'region': 'Champagne',
  

In [28]:
conda install pymongo

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [36]:
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
wine_db = client['wine_db']
wines = wine_db['wines']

In [68]:
for row in all_rows:
    wines.insert_one(row)